## Straturi Noi

In continuare o sa utilizam o parte din straturile prezentate in curs.

Staturi noi:

Layer Convolutional:
* [torch.nn.Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)(in_channels, out_channels, kernel_size, stride=1, padding=0)

Layere Pooling:
* [torch.nn.MaxPool2d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html)(kernel_size, stride=None, padding=0)
*  [torch.nn.AveragePool2d](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html)(kernel_size, stride=None, padding=0)

Layere Adaptive Pool, intalnit adesea si ca Global Pool:
* [torch.nn.AdaptiveAvgPool2d](https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveAvgPool2d.html)(output_size)
* [torch.nn.AdaptiveMaxPool2d](https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveMaxPool2d.html)(output_size)

Layer de liniarizare:

* [torch.nn.Flatten()](https://pytorch.org/docs/stable/generated/torch.flatten.html)



Four **hyperparameters** control the size of the output volume:
* **Depth**: number of filters, as each filter _looks_ at different areas of the input:
* **Stride**: the step taken when _sliding_ the filter. (Usually 1 or 2, 3 - uncommon).
* **Zero-Padding**: size of the number of 0s that surround the border of the input volume. Example: If you want to the same width and height for input and output.
* **Dilation**: Distance between elements of the convolutional kernel.



**Why Pooling Layer?**

1. Modifica volumul de input (input volume) in reprezentari _mai mici_ si mai usor de _manevrat_.
2. Opereaza independent pe fiecare Activation Map.

<img src="https://computersciencewiki.org/images/9/9e/MaxpoolSample.png" width="425" height="300"/> <img src="https://miro.medium.com/v2/resize:fit:517/0*lIcR0gOMK0xzTr6_.png" width="425" height="300"/>






**Why Adaptive Pooling Layer?**

1. Folosite de regula in etapele finale de constructie a unei arhitecturi de tipul _ConvNet_ pentru a inlocui fully-connected layers.
2. Incearca sa previna *overfitting phenomenon* fortand feature maps sa retina informatia **globala** care este relevanta pentru task-ul acestei _ConvNet_ (clasificare, identifcare etc.)

<img src="https://www.researchgate.net/publication/338079465/figure/fig4/AS:905983672987648@1593014748012/The-difference-of-max-pooling-and-global-max-pooling.ppm" width="725" height="300"/>


<img src="https://drive.google.com/uc?id=11l7Xsh-iQmASvXTkgH2MgtA01XCW6CAC">


[Visualise them Here](https://github.com/vdumoulin/conv_arithmetic).

In [1]:
import torch.multiprocessing as mp
mp.set_start_method('spawn')

In [2]:
import numpy as np
import torch.nn as nn
import torch

dummy_input_tensor = torch.rand((1,3,100,100))  # Input random de marime 100x100 cu 3 canale

layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(3,3), stride=(2,2))
print("Conv1 result shape",layer(dummy_input_tensor).shape)

layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(13,13), stride=(2,2))
print("Conv2 result shape",layer(dummy_input_tensor).shape)

layer = nn.MaxPool2d(kernel_size=(3,3)) # Stride este inferat din kernel size, ca fiind egal cu kernel size ca sa nu repete elementele luate
print("Pool result shape",layer(dummy_input_tensor).shape)

# Utilizate pentru a reduce dimensiunea la una prestabilita, util cand marimea input ului este variabil
layer = nn.AdaptiveAvgPool2d(output_size=(5,5))
print("Global Pool result shape",layer(dummy_input_tensor).shape)

layer = nn.Flatten()
print("Flaten result shape",layer(dummy_input_tensor).shape)

Conv1 result shape torch.Size([1, 10, 49, 49])
Conv2 result shape torch.Size([1, 10, 44, 44])
Pool result shape torch.Size([1, 3, 33, 33])
Global Pool result shape torch.Size([1, 3, 5, 5])
Flaten result shape torch.Size([1, 30000])


###Cerinte

**(1p)** Utilizati o serie de Conv2D/Pool2D pentru a ajunge la urmatoarele marimi plecand de la input 3x100x100:
*   [1, 10, 25, 25] # Stride & Padding
*   [1, 10, 32, 32]
*  [1, 3, 2, 2]



In [3]:
import numpy as np
import torch.nn as nn
import torch

dummy_input_tensor = torch.rand((1,3,100,100))  # Input random de marime 100x100 cu 3 canale

layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(3,3), stride=(4,4))
print("Conv1 result shape",layer(dummy_input_tensor).shape)

layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(7,7), stride=(3,3))
print("Conv2 result shape",layer(dummy_input_tensor).shape)

layer = nn.MaxPool2d(kernel_size=(50, 50))
print("Conv3 result shape",layer(dummy_input_tensor).shape)

Conv1 result shape torch.Size([1, 10, 25, 25])
Conv2 result shape torch.Size([1, 10, 32, 32])
Conv3 result shape torch.Size([1, 3, 2, 2])


## Instantierea seturilor de date

In [4]:
import torchvision

cifar_train = torchvision.datasets.CIFAR10("./data", download=True)
cifar_test = torchvision.datasets.CIFAR10("./data", train=False)

## Crearea Dataloader-ului

### Cerinte
 * **(2p)** Implementati functia de preprocesare a datelor, __collate_fn(examples)__.


Atentie! Spre deosebire de intrarea pentru retelele fully-connected, pentru retelele convolutionale intrearea nu trebuie liniarizata, ci doar normalizata.

#### Hint

  * Amintiti-va folosirea functiei __normalize__ din torchvision.transforms.functional din laboratorul trecut.
  * Modificati functia *collate_fn* din laboratorul trecut, pentru a normaliza datele in intervalul [-1, 1]

In [11]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision.transforms.functional import to_tensor, normalize

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_fn(examples):
  images, labels = zip(*examples)

  images = torch.stack([normalize(to_tensor(img), mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) for img in images])
  labels = torch.tensor(labels)

  return images.to(device), labels.to(device)


train_loader = DataLoader(cifar_train, batch_size=500, shuffle=True, num_workers=0, collate_fn=collate_fn)
test_loader = DataLoader(cifar_test, batch_size=1, shuffle=False, collate_fn=collate_fn)

## Crearea unei retele neurale convolutionale

### Cerinte
 * **(1p)** Creati o clasa care mosteneste clasa nn.Module. Ea va reprezenta o retea neurala convolutionala pentru clasificare ale celor 10 clase din datasetul CIFAR10.
    * Reteaua trebuie sa aiba 2 straturi convolutionale care sa reduca dimensiunea spatiala a imaginii de 2 ori (0.25 p).
    * Liniarizati iesirea din cel de-al doilea strat convolutional (0.25 p).
    * Adaugat stratul final de tipul `fully-connected` (0.25 p).
    * Folositi o functie de activare la alegere (Exemplu [RELU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)) (0.25 p).

#### Hint

Pentru a liniariza iesirea din cel de-al doilea feature map puteti adopta mai multe strategii:
  * Liniarizare prin schimbarea shape-ului la [batch_size, -1]
  * Global Max Pooling si apoi liniarizare la [batch_size, -1]
  * Average Max Pooling si apoi liniarizare la [batch_size, -1]

In [12]:
import torch.nn as nn

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()

    self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3,3), stride=(1,1))
    self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3,3), stride=(1,1))

    self.linear1 = nn.Linear(32 * 14 * 14, 10)

    self.activation = nn.ReLU()
    self.pool = nn.MaxPool2d(kernel_size=(2,2))

  def forward(self,x):
    # convolution
    x = self.activation(self.conv2(self.activation(self.conv1(x))))
    x = self.pool(x)

    # to not flattend the whole batch
    x = torch.flatten(x, start_dim=1)

    x = self.linear1(x)

    return x

## Definirea obiectelor folosite in timpul antrenarii

### Cerinte **(1p)**
  * Numarul de epoci (0.25 p)
  * Retea (0.25 p)
  * Optimizator (0.25 p)
  * Alegeti functia de cost (0.25 p)

In [14]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = 25

# Definiti reteaua
network = Net().to(device)

# Definiti optimizatorul
optimizer = optim.Adam(network.parameters(), lr=1e-2)
"""
Dupa definirea optimizatorului si dupa fiecare iteratie de antrenare, trebuie
apelata functia zero_grad() pentru a seta valoare tuturor gradientilor la zero.
"""
# Completati aici codul pentru seta valoare tuturor gradientilor la zero
optimizer.zero_grad()

# Definiti functia cost pentru clasificare Cross-Entropy
# https://neptune.ai/blog/pytorch-loss-functions
loss_fn = nn.CrossEntropyLoss()

## Definirea functiei de antrenare

In [15]:
from torch.utils.data import DataLoader

In [16]:
def test_acc(net: nn.Module, test_loader: DataLoader):
  net.eval()

  total = 0
  correct = 0

  for test_images, test_labels in test_loader:
    total += len(test_images)
    out_class = torch.argmax(net(test_images))
    correct += torch.sum(out_class == test_labels)

  return correct / total * 100


def train_fn(epochs: int, train_loader: DataLoader, test_loader: DataLoader,
             net: nn.Module, loss_fn: nn.Module, optimizer: optim.Optimizer):
  # Iteram prin numarul de epoci
  for e in range(epochs):
    net.train()

    # Iteram prin fiecare batch din dataloader
    for images, labels in train_loader:
      # Aplicam reteaua neurala pe imaginile din batch-ul curent
      out = net(images)
      # Aplicam functia cost pe iesirea retelei neurale si pe etichetele imaginilor
      loss = loss_fn(out, labels)
      # Aplicam algoritmul de back-propagation
      loss.backward()
      # Facem pasul de optimizare, pentru a actualiza parametrii retelei
      optimizer.step()
      # Apelam functia zero_grad() pentru a uita gradientii de la iteratie curenta
      optimizer.zero_grad()

    print("Loss-ul la finalul epocii {} are valoarea {}".format(e, loss.item()))

    # Calculam acuratetea
    acc = test_acc(net, test_loader)
    print("Acuratetea la finalul epocii {} este {:.2f}%".format(e + 1, acc))


## Antrenarea

### Cerinte
  * Antrenati reteaua definita mai sus (clasa Net)

In [17]:
train_fn(epochs, train_loader, test_loader, network, loss_fn, optimizer)

Loss-ul la finalul epocii 0 are valoarea 1.9859713315963745
Acuratetea la finalul epocii 1 este 26.85%
Loss-ul la finalul epocii 1 are valoarea 1.7949037551879883
Acuratetea la finalul epocii 2 este 32.80%
Loss-ul la finalul epocii 2 are valoarea 1.7026114463806152
Acuratetea la finalul epocii 3 este 40.00%
Loss-ul la finalul epocii 3 are valoarea 1.5770518779754639
Acuratetea la finalul epocii 4 este 45.58%
Loss-ul la finalul epocii 4 are valoarea 1.4993414878845215
Acuratetea la finalul epocii 5 este 48.43%
Loss-ul la finalul epocii 5 are valoarea 1.3488045930862427
Acuratetea la finalul epocii 6 este 49.63%
Loss-ul la finalul epocii 6 are valoarea 1.4058430194854736
Acuratetea la finalul epocii 7 este 50.15%
Loss-ul la finalul epocii 7 are valoarea 1.3080756664276123
Acuratetea la finalul epocii 8 este 50.22%
Loss-ul la finalul epocii 8 are valoarea 1.3928066492080688
Acuratetea la finalul epocii 9 este 50.70%
Loss-ul la finalul epocii 9 are valoarea 1.3575642108917236
Acuratetea la

## Reteaua LeNet

### Cerinte
  * **(3p)** Implementati reteaua LeNet dupa figura de mai jos si antrenati-o.



![alt text](https://drive.google.com/uc?id=1OVancUyIViMRMZdULFSVCvXJHQP0NGUV)

Figura arhitectura LeNet

![alt text](https://debuggercafe.com/wp-content/uploads/2019/07/Layers-in-LeNet.png)

Tabel arhitectura LeNet

_Question:_ Care este diferenta dintre `tanh` si `softmax`? De ce credeti ca peste ultimul layer (cel de output) a fost aplicata functia `softmax`?


**softmax** transforms data into a probabilty distribution, unlike **tanh** which transforms values in `[-1, 1]` range.

In [26]:
import torch.nn as nn

class LeNet(nn.Module):
  def __init__(self):
    super().__init__()
    """
    Punctaj: 2.5p
    """
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=(5,5), stride=1)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5,5), stride=1)

    self.linear1 = nn.Linear(in_features=16*5*5, out_features=120)
    self.linear2 = nn.Linear(in_features=120, out_features=84)
    self.linear3 = nn.Linear(in_features=84, out_features=10)

    self.pool = nn.AvgPool2d(kernel_size=(2,2), stride=2)
    self.tanh = nn.Tanh()
    self.softmax = nn.Softmax(dim=1)

  def forward(self,x):
    """
    Punctaj: 0.5p
    """
    x = self.pool(self.tanh(self.conv1(x)))
    x = self.pool(self.tanh(self.conv2(x)))

    x = torch.flatten(x, start_dim=1)

    x = self.tanh(self.linear1(x))
    x = self.tanh(self.linear2(x))
    x = self.softmax(self.linear3(x))

    return x

## Optional: Reteaua AlexNet ❤️

❗Daca alegeti aceasta retea veti continua sa rezolvati exercitiile urmatoare pentru reteaua AlexNet.

Pentru a usura volumul de munca si obtine o retea AlexNet comparabila in dificultate cu LeNet, urmati acesti pasi:

✔️ Includeti functii de activare intre layere (exemplu ReLU).

✔️ Va folositi doar de prima subsectiune din schema figurii arhitecturii AlexNet (adica doar Conv1 si Conv2 blocks).

✔️ Inputul vostru se opreste la un minimum size de 8x8.

✔️ Modificati output-ul retelei sa prezica 10 clase in loc de 1000 de clase.





![alt text](https://anhreynolds.com/img/alexnet.png)

Figura arhitectura AlexNet.

![alt text](https://anhreynolds.com/img/alexnet-parameters.png)

Tabel arhitectura AlexNet


In [ ]:
import torch.nn as nn

class AlexNet(nn.Module):
  def __init__(self):
    super().__init__()
    """
    Punctaj: 2.5p
    """
    pass

  def forward(self,x):
    """
    Punctaj: 0.5p
    """
    return x

## Redefinirea obiectelor folosite in timpul antrenarii pentru reteaua LeNet

### Cerinta
 * Redefiniti obiectele pentru a antrena reteaua LeNet

In [29]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = 20

# Definiti reteaua
lenet = LeNet().to(device)

# Definiti optimizatorul
lenet_optimizer = optim.Adam(lenet.parameters(), lr=1e-3)
# Dupa definirea optimizatorului si dupa fiecare iteratie trebuie apelata functia zero_grad().
# Aceasta face toti gradientii zero.
# Completati codul pentru a face gradientii zero aici
lenet_optimizer.zero_grad()


# Definiti functia cost pentru clasificare Cross-Entropy
loss_fn = nn.CrossEntropyLoss()

## Antrenarea retelei LeNet

In [30]:
train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer)

Loss-ul la finalul epocii 0 are valoarea 2.1221871376037598
Acuratetea la finalul epocii 1 este 33.17%
Loss-ul la finalul epocii 1 are valoarea 2.101656198501587
Acuratetea la finalul epocii 2 este 37.96%
Loss-ul la finalul epocii 2 are valoarea 2.064628839492798
Acuratetea la finalul epocii 3 este 41.12%
Loss-ul la finalul epocii 3 are valoarea 2.050738573074341
Acuratetea la finalul epocii 4 este 43.00%
Loss-ul la finalul epocii 4 are valoarea 1.998028039932251
Acuratetea la finalul epocii 5 este 44.50%
Loss-ul la finalul epocii 5 are valoarea 1.9625322818756104
Acuratetea la finalul epocii 6 este 45.51%
Loss-ul la finalul epocii 6 are valoarea 1.9630298614501953
Acuratetea la finalul epocii 7 este 46.64%
Loss-ul la finalul epocii 7 are valoarea 1.9607833623886108
Acuratetea la finalul epocii 8 este 48.14%
Loss-ul la finalul epocii 8 are valoarea 1.9576640129089355
Acuratetea la finalul epocii 9 este 48.33%
Loss-ul la finalul epocii 9 are valoarea 1.924136996269226
Acuratetea la fina

###Augmentare retea

Reteaua de mai devreme duce lipsa de regularizare. O forma foarte puternica de regularizare este normalizarea, iar pentru acest lucru exista straturi speciale.

Astfel de straturi:

* [torch.nn.BatchNorm2d](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html) (num_features)
* [torch.nn.InstanceNorm2d](https://pytorch.org/docs/stable/generated/torch.nn.InstanceNorm2d.html) (num_features)

Un alt element important il reprezinta functiile de activare, care pot influenta convergenta si puterea retelei. Cateva exemple de alte functii de activare:

* ReLU
* Sigmoid
* Tanh
* LeakyRelu
* GELU

## Cerinta

**(2p)** Experimentati cu aceste elemente in cadrul retelei LeNet definita mai devreme, pentru a obtine o acuratete mai buna. Observati viteza de convergenta si performanta retelei pentru 3 configuratii diferite.

**Punctaj:** 0.6p / configuratie.

0.6p din care:
- 0.4p modificarea retelei
- 0.1p obtinerea rezultatelor
- 0.1p afisarea acestora si explicatie.


###Bonus
**(1p)** Antrenati reteaua folosind GPU (Graphics processing unit)








  


# Configuration 01

- Added Relu
- Changed AvgPool to MaxPool

# Conclusion:

Faster convergence to optimum and better results, reached 61% accuracy rather than 51%

In [31]:
import torch.nn as nn
import torch.optim as optim


class LeNet(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=(5,5), stride=1)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5,5), stride=1)

    self.linear1 = nn.Linear(in_features=16*5*5, out_features=120)
    self.linear2 = nn.Linear(in_features=120, out_features=84)
    self.linear3 = nn.Linear(in_features=84, out_features=10)

    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=2)
    self.relu = nn.ReLU()

  def forward(self,x):
    x = self.pool(self.relu(self.conv1(x)))
    x = self.pool(self.relu(self.conv2(x)))

    x = torch.flatten(x, start_dim=1)

    x = self.relu(self.linear1(x))
    x = self.relu(self.linear2(x))
    x = self.linear3(x)

    return x


epochs = 20
lenet = LeNet().to(device)
lenet_optimizer = optim.Adam(lenet.parameters(), lr=1e-3)
lenet_optimizer.zero_grad()
loss_fn = nn.CrossEntropyLoss()

train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer)

Loss-ul la finalul epocii 0 are valoarea 1.7204524278640747
Acuratetea la finalul epocii 1 este 39.69%
Loss-ul la finalul epocii 1 are valoarea 1.5653542280197144
Acuratetea la finalul epocii 2 este 44.82%
Loss-ul la finalul epocii 2 are valoarea 1.3739248514175415
Acuratetea la finalul epocii 3 este 48.35%
Loss-ul la finalul epocii 3 are valoarea 1.3725754022598267
Acuratetea la finalul epocii 4 este 50.67%
Loss-ul la finalul epocii 4 are valoarea 1.3489803075790405
Acuratetea la finalul epocii 5 este 52.30%
Loss-ul la finalul epocii 5 are valoarea 1.3278206586837769
Acuratetea la finalul epocii 6 este 53.87%
Loss-ul la finalul epocii 6 are valoarea 1.2368723154067993
Acuratetea la finalul epocii 7 este 54.34%
Loss-ul la finalul epocii 7 are valoarea 1.214155912399292
Acuratetea la finalul epocii 8 este 55.19%
Loss-ul la finalul epocii 8 are valoarea 1.236330509185791
Acuratetea la finalul epocii 9 este 55.30%
Loss-ul la finalul epocii 9 are valoarea 1.1335994005203247
Acuratetea la f

# Configuration 02

- Added Batchnorm after every layer

# Conclusion:

Even faster convergence, but stopping at ~62%, a bit higher than the previous configuration.


In [34]:
import torch.nn as nn
import torch.optim as optim


class LeNet(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=(5,5), stride=1)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5,5), stride=1)

    self.linear1 = nn.Linear(in_features=16*5*5, out_features=120)
    self.linear2 = nn.Linear(in_features=120, out_features=84)
    self.linear3 = nn.Linear(in_features=84, out_features=10)

    self.bn1 = nn.BatchNorm2d(6)
    self.bn2 = nn.BatchNorm2d(16)
    self.bn3 = nn.BatchNorm1d(120)
    self.bn4 = nn.BatchNorm1d(84)

    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=2)
    self.relu = nn.ReLU()

  def forward(self,x):
    x = self.pool(self.bn1(self.relu(self.conv1(x))))
    x = self.pool(self.bn2(self.relu(self.conv2(x))))

    x = torch.flatten(x, start_dim=1)

    x = self.bn3(self.relu(self.linear1(x)))
    x = self.bn4(self.relu(self.linear2(x)))
    x = self.linear3(x)

    return x


epochs = 20
lenet = LeNet().to(device)
lenet_optimizer = optim.Adam(lenet.parameters(), lr=1e-3)
lenet_optimizer.zero_grad()
loss_fn = nn.CrossEntropyLoss()

train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer)

Loss-ul la finalul epocii 0 are valoarea 1.4120289087295532
Acuratetea la finalul epocii 1 este 50.68%
Loss-ul la finalul epocii 1 are valoarea 1.1876875162124634
Acuratetea la finalul epocii 2 este 56.91%
Loss-ul la finalul epocii 2 are valoarea 1.0439326763153076
Acuratetea la finalul epocii 3 este 58.57%
Loss-ul la finalul epocii 3 are valoarea 1.0150220394134521
Acuratetea la finalul epocii 4 este 60.92%
Loss-ul la finalul epocii 4 are valoarea 0.929127037525177
Acuratetea la finalul epocii 5 este 61.74%
Loss-ul la finalul epocii 5 are valoarea 0.9274863600730896
Acuratetea la finalul epocii 6 este 62.05%
Loss-ul la finalul epocii 6 are valoarea 0.8881296515464783
Acuratetea la finalul epocii 7 este 62.49%
Loss-ul la finalul epocii 7 are valoarea 0.981374979019165
Acuratetea la finalul epocii 8 este 62.23%
Loss-ul la finalul epocii 8 are valoarea 0.7251580953598022
Acuratetea la finalul epocii 9 este 62.99%
Loss-ul la finalul epocii 9 are valoarea 0.7379001379013062
Acuratetea la f

# Configuration 03

- Added Dropout after fully connected layers

# Conclusion:

The convergence slowed a bit, but the results are beter than before ~65%

In [35]:
import torch.nn as nn
import torch.optim as optim


class LeNet(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=(5,5), stride=1)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5,5), stride=1)

    self.linear1 = nn.Linear(in_features=16*5*5, out_features=120)
    self.linear2 = nn.Linear(in_features=120, out_features=84)
    self.linear3 = nn.Linear(in_features=84, out_features=10)

    self.bn1 = nn.BatchNorm2d(6)
    self.bn2 = nn.BatchNorm2d(16)
    self.bn3 = nn.BatchNorm1d(120)
    self.bn4 = nn.BatchNorm1d(84)

    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=2)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(p=0.5)

  def forward(self,x):
    x = self.pool(self.bn1(self.relu(self.conv1(x))))
    x = self.pool(self.bn2(self.relu(self.conv2(x))))

    x = torch.flatten(x, start_dim=1)

    x = self.bn3(self.relu(self.linear1(x)))
    x = self.dropout(x)

    x = self.bn4(self.relu(self.linear2(x)))
    x = self.dropout(x)

    x = self.linear3(x)

    return x


epochs = 20
lenet = LeNet().to(device)
lenet_optimizer = optim.Adam(lenet.parameters(), lr=1e-3)
lenet_optimizer.zero_grad()
loss_fn = nn.CrossEntropyLoss()

train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer)

Loss-ul la finalul epocii 0 are valoarea 1.6008747816085815
Acuratetea la finalul epocii 1 este 45.05%
Loss-ul la finalul epocii 1 are valoarea 1.4251517057418823
Acuratetea la finalul epocii 2 este 51.46%
Loss-ul la finalul epocii 2 are valoarea 1.3742750883102417
Acuratetea la finalul epocii 3 este 52.03%
Loss-ul la finalul epocii 3 are valoarea 1.2971786260604858
Acuratetea la finalul epocii 4 este 56.48%
Loss-ul la finalul epocii 4 are valoarea 1.2627116441726685
Acuratetea la finalul epocii 5 este 57.91%
Loss-ul la finalul epocii 5 are valoarea 1.127300500869751
Acuratetea la finalul epocii 6 este 59.43%
Loss-ul la finalul epocii 6 are valoarea 1.0609116554260254
Acuratetea la finalul epocii 7 este 60.66%
Loss-ul la finalul epocii 7 are valoarea 1.2251918315887451
Acuratetea la finalul epocii 8 este 61.17%
Loss-ul la finalul epocii 8 are valoarea 1.2392632961273193
Acuratetea la finalul epocii 9 este 62.29%
Loss-ul la finalul epocii 9 are valoarea 1.193796992301941
Acuratetea la f